In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../Base_de_datos/Ventas.xlsx')
df

,id_venta,fecha,id_cliente,nombre_cliente,email,medio_pago
0,1,2024-06-19,62,Guadalupe Romero,guadalupe.romero@mail.com,tarjeta
1,2,2024-03-17,49,Olivia Gomez,olivia.gomez@mail.com,qr
2,3,2024-01-13,20,Tomas Acosta,tomas.acosta@mail.com,tarjeta
3,4,2024-02-27,36,Martina Molina,martina.molina@mail.com,transferencia
4,5,2024-06-11,56,Bruno Diaz,bruno.diaz@mail.com,tarjeta
...,...,...,...,...,...,...
115,116,2024-03-18,25,Karina Castro,karina.castro@mail.com,qr
116,117,2024-03-14,72,Camila Rodriguez,camila.rodriguez@mail.com,tarjeta
117,118,2024-02-09,84,Pablo Sanchez,pablo.sanchez@mail.com,efectivo
118,119,2024-02-07,51,Agustina Gomez,agustina.gomez@mail.com,qr


# **1. Analizar problemas con el dataset**

### **Tipos de datos y valores nulos**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_venta        120 non-null    int64         
 1   fecha           120 non-null    datetime64[ns]
 2   id_cliente      120 non-null    int64         
 3   nombre_cliente  120 non-null    object        
 4   email           120 non-null    object        
 5   medio_pago      120 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 5.8+ KB


### **Estadísticas generales**

In [4]:
df.describe(include='all')

,id_venta,fecha,id_cliente,nombre_cliente,email,medio_pago
count,120.000000,120,120.000000,120,120,120
unique,NaN,NaN,NaN,64,67,4
top,NaN,NaN,NaN,Bruno Diaz,bruno.diaz@mail.com,efectivo
freq,NaN,NaN,NaN,5,5,37
mean,60.500000,2024-03-29 17:36:00,47.291667,NaN,NaN,NaN
min,1.000000,2024-01-02 00:00:00,1.000000,NaN,NaN,NaN
25%,30.750000,2024-02-11 06:00:00,24.500000,NaN,NaN,NaN
50%,60.500000,2024-03-25 00:00:00,48.500000,NaN,NaN,NaN
75%,90.250000,2024-05-19 06:00:00,67.500000,NaN,NaN,NaN
max,120.000000,2024-06-28 00:00:00,100.000000,NaN,NaN,NaN


### **Conteo de valores faltantes**

In [5]:
df.isnull().sum()

id_venta          0
fecha             0
id_cliente        0
nombre_cliente    0
email             0
medio_pago        0
dtype: int64

### **Conteo de filas duplicadas**

In [6]:
df.duplicated().sum()

np.int64(0)

### **Frecuencia por medio de pago**

In [7]:
df['medio_pago'].value_counts()

medio_pago
efectivo         37
qr               30
transferencia    27
tarjeta          26
Name: count, dtype: int64

### **Verificación de valores únicos en medios de pago**

In [8]:
df['medio_pago'].unique()

array(['tarjeta', 'qr', 'transferencia', 'efectivo'], dtype=object)

# **2. Limpieza de datos**

### **Eliminar columnas redundantes**

In [9]:
df = df.drop(['nombre_cliente', 'email'], axis=1)

### **Eliminación de filas duplicadas**

In [10]:
df = df.drop_duplicates()

### **Convertir fechas a formato datetime**

In [11]:
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

### **Quitar espacios y capitalizar**

In [12]:
df['medio_pago'] = df['medio_pago'].str.strip().str.title()

### **Eliminar filas con valores nulos en 'id_venta' o 'id_cliente'**

In [13]:
df = df.dropna(subset=['id_venta', 'id_cliente'])

# **Estandarización**

### **Asegurar que todos los datos sigan el mismo formato:**
- IDs con formato numérico correcto
- Medios de pago estandarizados
- Fechas en el mismo formato (YYYY-MM-DD)

In [14]:
df['id_venta'] = pd.to_numeric(df['id_venta'], errors='coerce')
df['id_cliente'] = pd.to_numeric(df['id_cliente'], errors='coerce')
df['medio_pago'] = df['medio_pago'].replace({
    'Tarjeta De Credito': 'Tarjeta de Crédito',
    'Tarjeta De Debito': 'Tarjeta de Débito',
    'Transferencia Bancaria': 'Transferencia Bancaria'
})
df['fecha'] = df['fecha'].dt.strftime('%Y-%m-%d')

# **3. Normalización de datos**

### **Normalización de IDs**

In [15]:
df['id_venta'] = pd.to_numeric(df['id_venta'], errors='coerce')
df['id_cliente'] = pd.to_numeric(df['id_cliente'], errors='coerce')

### **Normalización de fechas**

In [16]:
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df['año_venta'] = df['fecha'].dt.year
df['mes_venta'] = df['fecha'].dt.month

### **One-hot encoding para medios de pago**

In [17]:
df = pd.get_dummies(df, columns=['medio_pago'], prefix='pago')

df

,id_venta,fecha,id_cliente,año_venta,mes_venta,pago_Efectivo,pago_Qr,pago_Tarjeta,pago_Transferencia
0,1,2024-06-19,62,2024,6,False,False,True,False
1,2,2024-03-17,49,2024,3,False,True,False,False
2,3,2024-01-13,20,2024,1,False,False,True,False
3,4,2024-02-27,36,2024,2,False,False,False,True
4,5,2024-06-11,56,2024,6,False,False,True,False
...,...,...,...,...,...,...,...,...,...
115,116,2024-03-18,25,2024,3,False,True,False,False
116,117,2024-03-14,72,2024,3,False,False,True,False
117,118,2024-02-09,84,2024,2,True,False,False,False
118,119,2024-02-07,51,2024,2,False,True,False,False


In [19]:
# Convertir las columnas de dummies a tipo entero

dummies_cols = ['pago_Efectivo', 'pago_Qr',	'pago_Tarjeta',	'pago_Transferencia']
df[dummies_cols] = df[dummies_cols].astype(int)

df

,id_venta,fecha,id_cliente,año_venta,mes_venta,pago_Efectivo,pago_Qr,pago_Tarjeta,pago_Transferencia
0,1,2024-06-19,62,2024,6,0,0,1,0
1,2,2024-03-17,49,2024,3,0,1,0,0
2,3,2024-01-13,20,2024,1,0,0,1,0
3,4,2024-02-27,36,2024,2,0,0,0,1
4,5,2024-06-11,56,2024,6,0,0,1,0
...,...,...,...,...,...,...,...,...,...
115,116,2024-03-18,25,2024,3,0,1,0,0
116,117,2024-03-14,72,2024,3,0,0,1,0
117,118,2024-02-09,84,2024,2,1,0,0,0
118,119,2024-02-07,51,2024,2,0,1,0,0


# **4. Validación Post-Limpieza y Normalización**

### **Verificar el resultado de la limpieza**

In [20]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_venta            120 non-null    int64         
 1   fecha               120 non-null    datetime64[ns]
 2   id_cliente          120 non-null    int64         
 3   año_venta           120 non-null    int32         
 4   mes_venta           120 non-null    int32         
 5   pago_Efectivo       120 non-null    int64         
 6   pago_Qr             120 non-null    int64         
 7   pago_Tarjeta        120 non-null    int64         
 8   pago_Transferencia  120 non-null    int64         
dtypes: datetime64[ns](1), int32(2), int64(6)
memory usage: 7.6 KB


,id_venta,fecha,id_cliente,año_venta,mes_venta,pago_Efectivo,pago_Qr,pago_Tarjeta,pago_Transferencia
0,1,2024-06-19,62,2024,6,0,0,1,0
1,2,2024-03-17,49,2024,3,0,1,0,0
2,3,2024-01-13,20,2024,1,0,0,1,0
3,4,2024-02-27,36,2024,2,0,0,0,1
4,5,2024-06-11,56,2024,6,0,0,1,0


# **5. Exportar datos limpios y normalizados**

In [21]:
df.to_csv('../Base_de_datos_limpia/Ventas_limpio.csv', index=False)